In [1]:
Div = (1<<8) + (1<<4) + (1<<3) + 2 + 1
global sbox
sbox = [[0]*16 for x in range(16)]
def add(a,b):
    return a^b

def mult(a,b):
    out = 0
    while(b!=0):
        if((b&1)!=0):
            out=add(out,a)
        b=b>>1
        a=a<<1
    return out


def mod(a,b):
    abits = 0
    bbits = 0
    tmp = a
    while(tmp!=0):
        abits+=1
        tmp = tmp>>1
    tmp = b
    while(tmp!=0):
        bbits+=1
        tmp = tmp>>1
    div = b
    while(bbits<abits):
        bbits+=1
        div = div<<1
        
    
    while(div>=b):
        if(a & (1<<(bbits-1))):
            a = a^div
        div=div>>1
        bbits-=1
    return a
    
def inverseMod(a):
    for i in range(256):
        if(mod(mult(a,i),Div) == 1):
            return i
    
    return 0

def sboxByte(a):
    invA = inverseMod(a)
    b = []
    bp = []
    x = [1,1,0,0,0,1,1,0]
    for i in range(8):
        if(invA & (1<<i)):
            b.append(1)
        else:
            b.append(0)
    for i in range(8):
        bp.append(b[i] ^ b[(i+4)%8] ^ b[(i+5)%8] ^ b[(i+6)%8] ^ b[(i+7)%8] ^ x[i])
    sboxVal = 0
    for i in range(8):
        sboxVal = sboxVal<<1
        sboxVal += bp[7-i]
    sbox[a>>4][a%(1<<4)] = sboxVal
def fillSBox():
    for i in range(256):
        sboxByte(i)
fillSBox()
# print(sbox)




In [2]:
global inputMat
inputMat = [[0]*4 for x in range(4)]
def textToMat(s):
    newMat = [[0]*4 for x in range(4)]
    inFile = open(s,"r")
    text = inFile.read()
    row = 0
    col = 0
    for i in range(16):
        newMat[row][col] = ord(text[i])
        col+=1
        if(col==4):
            row+=1
            col=0
            
    return newMat


    

def substitution(newMat):
    for i in range(4):
        for j in range(4):
            tmp = newMat[i][j]
# #             print(str(tmp>>4) + " " + str(tmp % (1<<4)))
            newMat[i][j] = sbox[tmp>>4][tmp % (1<<4)]
    return newMat
    
def shiftRow(newMat):
    for i in range(3):
        newMat[i+1] = newMat[i+1][i+1:]+newMat[i+1][:i+1]
        
    return newMat
        
# inputMat = textToMat("input.txt")
# print(inputMat)
# inputMat = putSubByte(inputMat)
# print(inputMat)
# rowTransform(inputMat)
# print(inputMat)
    

In [3]:
global mixer
mixer = [[2,3,1,1],[1,2,3,1],[1,1,2,3],[3,1,1,2]]
def mixerMult(a,b):
    x=[0,0,0,1,1,0,1,1]
    if(b == 1):
        return a;
    elif(b == 2):
        a = a<<1
        if(a & (1<<8)):
            a = a%(1<<8)
            for i in range(len(x)):
                a=a^(x[i]<<(len(x)-1-i))
#                 print(str(i) + " " + str(len(x)-1-i))
                
        return a
    elif(b == 3):
        return a^mixerMult(a,2)

def mixerMatMult(q,p):
    multMat = [[0]*4 for x in range(4)]
    for i in range(4):
        for j in range(4):
            multMat[i][j] = 0
            for k in range(4):
                multMat[i][j] = multMat[i][j] ^ mixerMult(p[k][j],q[i][k])
                
    return multMat


def mixColoumn(newMat):
    return mixerMatMult(mixer,newMat)
# inputMat = mixerMatMult(mixer,inputMat)
# print(inputMat)

In [4]:
global keyMat
keyMat = [[0]*4 for x in range(4)]


def addRoundKey(mat,key):
    for i in range(4):
        for j in range(4):
            mat[i][j] = mat[i][j] ^ key[i][j]
            
    return mat

# inputMat = addRoundKey(inputMat,keyMat)
# print(inputMat)

In [5]:
import math
global rc

    
rc = [0,1,2,4,8,10,20,40,80,27,54]

def RotWord(a):
    tmp = a>>24
    a = a % (1<<24)
    a = a<<8
    a = a ^ tmp
    return a

def Subword(a):
    w = 0
    for i in range(4):
        x = (a % (1<<8))
        a = a>>8
        nx = sbox[x>>4][x % (1<<4)]
        nx = nx << (i*8)
        w = w^nx
        
    return w
    
def keyExpansion(key, word):
    for i in range(4):
        word[i]=(key[4*i]<<24)+ (key[4*i+1]<<16) + (key[4*i+2]<<8) + key[4*i+3]
        
    for i in range(4,44):
        tmp = word[i-1]
        if((i%4)==0):
            tmp = Subword(RotWord(tmp))
            tmp = tmp ^ (rc[math.floor(i/4)]<<24)
        word[i] = word[i-4]^tmp
        
    return word

def getKeyfromExpansion(word, epoch):
    row = 0
    col = 0
    Key = [[0]*4 for x in range(4)]
    for i in range(4):
        Key[0][i] = (word[epoch +i] >> 24) % (1<<8)
        Key[1][i] = (word[epoch +i] >> 16) % (1<<8)
        Key[2][i] = (word[epoch +i] >> 8) % (1<<8)
        Key[3][i] = (word[epoch +i] >> 0) % (1<<8)
    
    return Key
            
    

# wordSet = keyExpansion(keyText, wordSet)
# print(wordSet)

def strToMat(str) :
    ret = [[],[],[],[]]
    for i in range(0,len(str),2) :
        cur = "" + str[i] + str[i+1]
        ret[(int(i/2))%4].append(int(cur,16))
    return ret
hexa = ['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f']
def MatToStr(mat) :
    ret = ""
    for j in range(0,4):
        for i in range(0,4):
            ret += hexi[int(mat[i][j]/16)] + hexi[mat[i][j]%16]
    return ret


def calcDef(mat1, mat2):
    sum = 0
    for i in range(4):
        for j in range(4):
            ret = mat1[i][j] ^ mat2[i][j]
            sum += bin(ret).count("1")
            
    return sum


def buildLinearKey(key):
    sz=0
    linearKey=[0]*16
    for i in range(4):
        for j in range(4):
            linearKey[sz]=key[j][i]
            sz=sz+1
            
    return linearKey

In [6]:
plaintext1  = "0123456789abcdeffedcba9876543210"
plaintext2  = "0023456789abcdeffedcba9876543210"
key1 = "0f1571c947d9e8590cb7add6af7f6798"
key2 = "0e1571c947d9e8590cb7add6af7f6798"

In [7]:
# buildInputMat("input.txt")
# buildKeyMat("key.txt")
text1 = strToMat(plaintext1)
text2 = strToMat(plaintext2)
# print(MatToStr(text1))
nkey = strToMat(key1)
lkey = buildLinearKey(nkey)
wordSet = [0]*44
# print(nkey)
wordSet = keyExpansion(lkey, wordSet)
cnt = 0
text1 = addRoundKey(text1,nkey)
text2 = addRoundKey(text2,nkey)
print("Epoch 1: ",MatToStr(text1),MatToStr(text2),calcDef(text1,text2))
cnt += 4
nkey = getKeyfromExpansion(wordSet, cnt)
# print("1: ",MatToStr(nkey))
# print(nkey)
for epoch in range(1,8):
    text1 = substitution(text1)
    text1 = shiftRow(text1)
    text1 = mixColoumn(text1)
    text1 = addRoundKey(text1,nkey)
    
    text2 = substitution(text2)
    text2 = shiftRow(text2)
    text2 = mixColoumn(text2)
    text2 = addRoundKey(text2,nkey)
    
    print("Epoch "+str(epoch)+": ",MatToStr(text1),MatToStr(text2),calcDef(text1,text2))
    
    cnt += 4
    nkey = getKeyfromExpansion(wordSet, cnt)
#     print(MatToStr(nkey))
    
text1 = substitution(text1)
text1 = shiftRow(text1)
text1 = addRoundKey(text1,nkey)

text2 = substitution(text2)
text2 = shiftRow(text2)
text2 = addRoundKey(text2, nkey)
print("Epoch "+str(epoch+1)+": ",MatToStr(text1),MatToStr(text2),calcDef(text1,text2))

Epoch 1:  0e3634aece7225b6f26b174ed92b5588 0f3634aece7225b6f26b174ed92b5588 1
Epoch 1:  657470750fc7ff3fc0e8e8ca4dd02a9c c4a9ad090fc7ff3fc0e8e8ca4dd02a9c 20
Epoch 2:  5c7bb49a6b72349b05a2317ff46d1294 fe2ae569f7ee8bb8c1f5a2bb37ef53d5 58
Epoch 3:  7115262448dc747e5cdac7227da9bd9c ec093dfb7c45343d689017507d485e62 59
Epoch 4:  f867aee8b437a5210c24c1974cffeabc 43efdb697244df808e8d9364ee0ae6f5 61
Epoch 5:  681eb200a006206dd1d4bce71efa654e 6128a5d5f7643287fa06c099a1375302 68
Epoch 6:  2edcddc195ede3dfade4d6eaaa657b17 920c166737b0b71b53fa6d0f0e74c0f1 69
Epoch 7:  ea7e215e7fde44b4dd11dab7db7211a2 406b9106aead8141e06c390414a70d92 69
Epoch 7:  593464b4c5967ca4b28c21c955424c22 d7bc21c1f344299392905dc716ce5b79 58


In [8]:
# buildInputMat("input.txt")
# buildKeyMat("key.txt")
text = strToMat(plaintext1)
# print(MatToStr(text1))
nkey1 = strToMat(key1)
nkey2 = strToMat(key2)
lkey1 = buildLinearKey(nkey1)
lkey2 = buildLinearKey(nkey2)
wordSet1 = [0]*44
wordSet2 = [0]*44
# print(nkey)
wordSet1 = keyExpansion(lkey1, wordSet1)
wordSet2 = keyExpansion(lkey2, wordSet2)
cnt = 0
text1 = addRoundKey(text,nkey1)
text2 = addRoundKey(text,nkey2)
print("Epoch 1: ",MatToStr(text1),MatToStr(text2),calcDef(text1,text2))
cnt += 4
nkey1 = getKeyfromExpansion(wordSet1, cnt)
nkey2 = getKeyfromExpansion(wordSet2, cnt)
# print("1: ",MatToStr(nkey))
# print(nkey)
for epoch in range(1,8):
    text1 = substitution(text1)
    text1 = shiftRow(text1)
    text1 = mixColoumn(text1)
    text1 = addRoundKey(text1,nkey1)
    
    text2 = substitution(text2)
    text2 = shiftRow(text2)
    text2 = mixColoumn(text2)
    text2 = addRoundKey(text2,nkey2)
    
    print("Epoch "+str(epoch)+": ",MatToStr(text1),MatToStr(text2),calcDef(text1,text2))
    
    cnt += 4
    nkey1 = getKeyfromExpansion(wordSet1, cnt)
    nkey2 = getKeyfromExpansion(wordSet2, cnt)
#     print(MatToStr(nkey))
    
text1 = substitution(text1)
text1 = shiftRow(text1)
text1 = addRoundKey(text1,nkey1)

text2 = substitution(text2)
text2 = shiftRow(text2)
text2 = addRoundKey(text2, nkey2)

print("Epoch "+str(epoch)+": ",MatToStr(text1),MatToStr(text2),calcDef(text1,text2))

Epoch 1:  0023456789abcdeffedcba9876543210 0023456789abcdeffedcba9876543210 0
Epoch 1:  82fa800cf9192caf2b684f02d96fa0f8 390343c9ee9a5c2873aca8676a3d4d05 71
Epoch 2:  56bf7c50b156aaac70f195fefd74b02b 8d86e3388f72ace09da506aba050aaf5 64
Epoch 3:  aae883934edf22c4d99ee64e16beb7f5 3225ff33cfb88656ee7b686a550c97ea 57
Epoch 4:  beb861d993761ce68163dfb5c7674dcf 1d8de0bd8299f0321c044379a7a29030 69
Epoch 5:  597002ab84d49d70f8755e943c34abad 6bb696f59bd8452ef75d81cf7e9f85ff 63
Epoch 6:  c7e1db3dba93d4fd215cabc4b92cc91a 551aa2913583281ce7ba345030f24d5d 68
Epoch 7:  59759ae51e06bbacfb433bc4740491bd a42d7a2dd6e7dddd7197c3840d2987f3 60
Epoch 7:  1546d1f4650e7f257c3e64d57e2cbd97 da872a88e1a6e92fd05331853b5396d4 64
